In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

VER = 200
USE = 'orders'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
users = pd.read_parquet(f'../../data/train_with_features/train_v172_X.pqt')
print( users.shape )
users.head()

(13465600, 226)


,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2,ts_max7,ts_min7,ts_diff7,z1,z2,z3,z4,z5,z6,buy_ratio7,count_item7,count_user7,hour_mean7,day_mean7,repeat7,buy_ratio_std7,hour_std7,day_std7,prev7,next7,orders7,carts7,order_repeat7,cart_repeat7,buy_ratio8,count_item8,count_user8,hour_mean8,day_mean8,repeat8,buy_ratio_std8,hour_std8,day_std8,prev8,next8,orders8,carts8,order_repeat8,cart_repeat8,buy_ratio9,count_item9,count_user9,hour_mean9,day_mean9,repeat9,buy_ratio_std9,hour_std9,day_std9,prev9,next9,orders9,carts9,order_repeat9,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10,z10,z11,buy_ratio11,count_item11,count_user11,hour_mean11,day_mean11,repeat11,buy_ratio_std11,hour_std11,day_std11,buy_ratio12,count_item12,count_user12,hour_mean12,day_mean12,repeat12,buy_ratio_std12,hour_std12,day_std12,buy_ratio13,count_item13,count_user13,hour_mean13,day_mean13,repeat13,buy_ratio_std13,hour_std13,day_std13,buy_ratio14,count_item14,unique_item14,hour_mean14,day_mean14,repeat14,buy_ratio_std14,hour_std14,day_std14,buy_ratio15,count_item15,unique_item15,hour_mean15,day_mean15,repeat15,buy_ratio_std15,hour_std15,day_std15,last_0,last_1,last_2,last_3,last_4,first_0,first_1,first_2,first_3,first_4,buy_0,buy_1,buy_2,buy_3,buy_4,e_mean_0,e_std_0,e_mean_1,e_std_1,e_mean_2,e_std_2,e_mean_3,e_std_3,click_oof
0,11098528,11830,0,-1.000000,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.514214,-1.0,0.671773,-1.0,-1.000,1.414214,1.514214,1.414214,1.414214,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.188454,14481.0,8606.0,1.682663,0.474975,643.345764,643.345764,0.132450,0.173267,0.148649,0.135000,0.115385,0.097744,0.061404,151.0,202.0,222.0,200.0,208.0,133.0,114.0,112.0,130.0,145.0,142.0,156.0,99.0,80.0,42.248135,42.248135,0.175659,32546.0,18362.0,49847.566406,308206.03125,1.772465,0.459607,17870.607422,193386.031250,0.129268,1230.0,849.0,50060.824219,313896.90625,1.448763,0.372407,17396.035156,186814.156250,369.0,372.0,16.0,127.0,1.066667,1.067227,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,0,0,0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,0.163253,8153.0,5035.0,48766.132812,309523.93750,1.619265,0.450127,17233.292969,181986.593750,2183.0,2087.0,269.0,793.0,3.086957,2.235294,0.154491,16700.0,10043.0,64008.941406,322344.93750,1.662850,0.430123,10567.528320,193000.531250,4227.0,3740.0,454.0,1672.0,2.224138,2.139628,0.197968,15846.0,9400.0,34922.984375,293305.12500,1.685745,0.487779,10204.664062,192677.828125,4013.0,3537.0,627.0,1883.0,1.958491,1.965084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.233175,4636.0,2785.0,47927.398438,308860.62500,1.664632,0.422898,18213.400391,198956.890625,1.111888,143.0,122.0,48456.414062,334845.21875,1.172131,0.316337,15841.716797,195765.437500,1.111888,143.0,122.0,48456.414062,334845.21875,1.172131,0.316337,15841.716797,195765.4375,0.0,1.0,1.0,79200.0,338400.0,1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,-1.0,-1.0,-1.0,-1.0,1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,0.0,1.000000,0.0,1.

# CatBoost

In [4]:
import catboost as cat
from sklearn.model_selection import GroupKFold
from catboost import CatBoostRanker, CatBoostClassifier, Pool
print('CatBoost version',cat.__version__)

FOLDS=5
SEED = 42

CatBoost version 1.0.6


In [5]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

223
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [6]:
%%time
if 0:
    for c in users.columns[2:]:
        print(c,', ',end='')
        if str(users[c].dtype)=='float32':
            users[c] = users[c].astype('float16')
        if str(users[c].dtype)=='int32':
            users[c] = users[c].astype('int16')

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 12.4 µs


In [7]:
# RMSE QueryRMSE PairLogit, PairLogitPairwise, YetiRank, YetiRankPairwise

In [8]:
%%time
oof = np.zeros(len(users))

skf = GroupKFold(n_splits=FOLDS)
for fold,(train_idx, valid_idx) in enumerate(skf.split(
            X=users, y=users[USE], groups=users.user )):
    
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print('#'*25)
    
    model = CatBoostRanker(iterations=5_000,
                              random_state=SEED,
                              max_depth=4,
                              #max_leaves=32,
                              learning_rate=0.1,
                              #subsample=0.7,
                              task_type='GPU',
                              #custom_metric = 'MAP:top=50',
                              #eval_metric = 'MAP',
                              #colsample_bylevel=0.5,
                              #thread_count=16,
                              #scale_pos_weight=10
                              loss_function='PairLogitPairwise'
                           )
    
    train = Pool(
        data = users.loc[train_idx, FEATURES],
        label = users.loc[train_idx, USE],
        group_id = users.loc[train_idx, 'user']
    )
    
    valid = Pool(
        data = users.loc[valid_idx, FEATURES],
        label = users.loc[valid_idx, USE],
        group_id = users.loc[valid_idx, 'user']
    )
    
    model.fit(train,
             verbose=100,
             early_stopping_rounds=250,
             eval_set=valid,
             )
    model.save_model(f'../../data/models/CAT_fold{fold}_{USE}_v{VER}.cat')
    

    if 0:
        feature_importance = model.feature_importances_
        sorted_idx = np.argsort(feature_importance)
        fig = plt.figure(figsize=(10, 20))
        plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
        plt.yticks(range(len(sorted_idx)), np.array(FEATURES)[sorted_idx])
        plt.title('Feature Importance')
        plt.show()
          
    # INFER OOF FOLD K
    oof_preds = model.predict(users.loc[valid_idx, FEATURES])
    oof[valid_idx] = oof_preds
    
    del model
    gc.collect()

#########################
### Fold 1
### Train size 10772400 Valid size 2693200
#########################
0:	learn: 0.6161213	test: 0.6166143	best: 0.6166143 (0)	total: 28.1ms	remaining: 2m 20s
100:	learn: 0.1036304	test: 0.1074041	best: 0.1074041 (100)	total: 2.5s	remaining: 2m 1s
200:	learn: 0.0994734	test: 0.1036372	best: 0.1036372 (200)	total: 4.91s	remaining: 1m 57s
300:	learn: 0.0974781	test: 0.1022662	best: 0.1022662 (300)	total: 7.31s	remaining: 1m 54s
400:	learn: 0.0959802	test: 0.1014927	best: 0.1014927 (400)	total: 9.75s	remaining: 1m 51s
500:	learn: 0.0947563	test: 0.1009454	best: 0.1009454 (500)	total: 12.2s	remaining: 1m 49s
600:	learn: 0.0936900	test: 0.1005911	best: 0.1005911 (600)	total: 14.6s	remaining: 1m 47s
700:	learn: 0.0927115	test: 0.1003473	best: 0.1003432 (699)	total: 17.1s	remaining: 1m 44s
800:	learn: 0.0918031	test: 0.1001888	best: 0.1001788 (786)	total: 19.5s	remaining: 1m 42s
900:	learn: 0.0909524	test: 0.1000243	best: 0.1000220 (899)	total: 21.9s	remaini

200:	learn: 0.1000926	test: 0.1009484	best: 0.1009484 (200)	total: 4.87s	remaining: 1m 56s
300:	learn: 0.0980583	test: 0.0997366	best: 0.0997366 (300)	total: 7.29s	remaining: 1m 53s
400:	learn: 0.0966086	test: 0.0989943	best: 0.0989943 (400)	total: 9.86s	remaining: 1m 53s
500:	learn: 0.0953452	test: 0.0984433	best: 0.0984433 (500)	total: 12.6s	remaining: 1m 53s
600:	learn: 0.0942810	test: 0.0981213	best: 0.0981213 (600)	total: 15.4s	remaining: 1m 52s
700:	learn: 0.0933205	test: 0.0979342	best: 0.0979342 (700)	total: 18.3s	remaining: 1m 51s
800:	learn: 0.0923885	test: 0.0976873	best: 0.0976764 (795)	total: 20.9s	remaining: 1m 49s
900:	learn: 0.0915390	test: 0.0975120	best: 0.0975049 (899)	total: 23.6s	remaining: 1m 47s
1000:	learn: 0.0907070	test: 0.0973467	best: 0.0973467 (1000)	total: 26.4s	remaining: 1m 45s
1100:	learn: 0.0899305	test: 0.0973280	best: 0.0973183 (1034)	total: 29.3s	remaining: 1m 43s
1200:	learn: 0.0891817	test: 0.0972606	best: 0.0972551 (1194)	total: 32.2s	remaining: 

# Make Sub

In [9]:
sub = users[['user','item']].copy()
sub['p'] = oof
#sub = cudf.DataFrame(sub)
sub = sub.sort_values(['user','p'],ascending=[True,False])
print( sub.shape )
sub.head()

(13465600, 3)


,user,item,p
0,11098528,11830,5.487175
1,11098528,1732105,3.839515
2,11098528,588923,2.614021
6,11098528,876129,2.324099
3,11098528,1157882,1.735687


In [10]:
sub = sub.reset_index(drop=True)

In [11]:
sub['n'] = sub.groupby('user').item.cumcount().astype('int8')

In [12]:
sub.head()

,user,item,p,n
0,11098528,11830,5.487175,0
1,11098528,1732105,3.839515,1
2,11098528,588923,2.614021,2
3,11098528,876129,2.324099,3
4,11098528,1157882,1.735687,4


In [13]:
sub = sub.loc[sub.n<20]
print( sub.shape )

(2693120, 4)


In [14]:
#sub = sub.to_pandas()

In [15]:
%%time
sub2 = sub.groupby('user').item.apply(list)

CPU times: user 2.24 s, sys: 58.5 ms, total: 2.3 s
Wall time: 2.29 s


In [16]:
sub2 = sub2.to_frame().reset_index()

In [17]:
%%time
sub2.item = sub2.item.apply(lambda x: " ".join(map(str,x)))

CPU times: user 514 ms, sys: 15.2 ms, total: 530 ms
Wall time: 528 ms


In [18]:
sub2.columns = ['session_type','labels']

In [19]:
sub2.session_type = sub2.session_type.astype('str')+ f'_{USE}'

In [20]:
sub2.head()

,session_type,labels
0,11098528_orders,11830 1732105 588923 876129 1157882 571762 307...
1,11098530_orders,409236 264500 1603001 364155 963957 583026 294...
2,11098531_orders,1557766 452188 1365569 1309633 1449555 1271998...
3,11098533_orders,1074173 1309900 1165015 765030 935297 833149 8...
4,11098534_orders,223062 1449202 908024 1342293 1607945 1300062 ...


In [21]:
submission = sub2

In [22]:
def fill_up(x):
    return []

In [23]:
%%time
submission['session'] = submission.session_type.apply(lambda x: int(x.split('_')[0]))
submission['type'] = submission.session_type.apply(lambda x: x.split('_')[1])
submission.labels = submission.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

test_labels = pd.read_parquet('/raid/Kaggle/otto/valid/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==USE]

test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
test_labels.loc[test_labels.labels.isna(),'labels'] =\
    test_labels.loc[test_labels.labels.isna(),'labels'].map(fill_up)

test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)

recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 

score = (recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()
print( score , recall_per_type)

# orders    0.669116

0.40184485944915943 type
orders    0.669741
dtype: float64
CPU times: user 5.81 s, sys: 255 ms, total: 6.07 s
Wall time: 5.81 s
